In [1]:
import numpy as np
import pandas as pd
!pip install lxml

# PART 1 OF ASSIGNMENT

In [33]:
df_T=pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')

### finding the correct dataframe in the list

In [35]:
df_T=df_T[0]

In [36]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Ignore cells with a borough that are Not assigned

In [37]:
df_T=df_T[df_T.Borough !='Not assigned']
df_T.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


### combine rows where more than one neighbourhood has the same postal code area

In [38]:
df_T=df_T.groupby(['Postcode','Borough'])['Neighbourhood'].agg(','.join)

In [39]:
df_T=df_T.sample(frac=1).reset_index()

In [45]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M4C,East York,Woodbine Heights
1,M5N,Central Toronto,Roselawn
2,M6P,West Toronto,"High Park,The Junction South"
3,M3M,North York,Downsview Central
4,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market"


### If a cell has a borough but a Not assigned neighbourhood then the neighbourhood will be equal the borough

In [50]:
df_T['Neighbourhood']=df_T.apply(
    lambda row: row['Borough'] if row['Neighbourhood']=='Not assigned' else row['Neighbourhood'],
    axis=1)

In [51]:
df_T.head()

,Postcode,Borough,Neighbourhood
0,M4C,East York,Woodbine Heights
1,M5N,Central Toronto,Roselawn
2,M6P,West Toronto,"High Park,The Junction South"
3,M3M,North York,Downsview Central
4,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market"


In [52]:
df_T.shape

(103, 3)

# PART 2 OF ASSIGNMENT

In [64]:
dfcoordinates=pd.read_csv('Geospatial_Coordinates.csv')

In [65]:
dfcoordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### creating a new dataframe with coordinates

In [70]:
dfcoordinates=dfcoordinates.rename(columns={"Postal Code":"Postcode"})

In [72]:
dfmerged=pd.merge(df_T, dfcoordinates, on='Postcode')

In [73]:
dfmerged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4C,East York,Woodbine Heights,43.695344,-79.318389
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763
3,M3M,North York,Downsview Central,43.728496,-79.495697
4,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049


# PART 3 OF ASSIGNMENT

In [74]:
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    scipy-1.3.2                |   py36h921218d_0        18.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    nu

### creating a new dataframe so that we only work with boroughs that contain the word Toronto

In [80]:
toronto_data = dfmerged[dfmerged['Borough'].str.contains("Toronto")].reset_index(drop=True)

In [83]:
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936
1,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763
2,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817
4,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450


### finding the coordinates of Toronto

In [84]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


### Creating a map of Toronto with toronto_data superimposed on top

In [86]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### defining foursquare credentials

In [87]:
CLIENT_ID = '0C4DJQEIFY21FWU3MME5PAD5ITOVOWJBASEL44WRC4XRS3W2' # your Foursquare ID
CLIENT_SECRET = 'TQ0ZIVSW3XMTPFAA5ZYB2GNMYYCUNLAY0B02BRXUYMWWILBY' # your Foursquare Secret
VERSION = '20180605'

### exploring the neighbourhoods in Toronto

In [106]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### creating a new dataframe called toronto_venues

In [107]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Roselawn
High Park,The Junction South
Chinatown,Grange Park,Kensington Market
Commerce Court,Victoria Hotel
Runnymede,Swansea
The Beaches
Harbourfront East,Toronto Islands,Union Station
Ryerson,Garden District
Adelaide,King,Richmond
Cabbagetown,St. James Town
Central Bay Street
The Danforth West,Riverdale
Berczy Park
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Harbourfront
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
Church and Wellesley
Dovercourt Village,Dufferin
First Canadian Place,Underground city
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Design Exchange,Toronto Dominion Centre
Harbord,University of Toronto
Lawrence Park
Christie
Stn A PO Boxes 25 The Esplanade
Moore Park,Summerhill East
St. James Town
Forest Hill North,Forest Hill West
North Toronto West
The Beaches West,India Bazaar
Parkdale,Roncesvalles
Studio District
The Annex,North Midtown,Yorkville
Rosedale
Davisville North
Da

In [108]:
print(toronto_venues.shape)
toronto_venues.head()

(1686, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Roselawn,43.711695,-79.416936,Rosalind's Garden Oasis,43.712189,-79.411978,Garden
1,"High Park,The Junction South",43.661608,-79.464763,Lithuania Park,43.658667,-79.463038,Park
2,"High Park,The Junction South",43.661608,-79.464763,nodo,43.665303,-79.465621,Italian Restaurant
3,"High Park,The Junction South",43.661608,-79.464763,Hole in the Wall,43.665296,-79.465118,Bar
4,"High Park,The Junction South",43.661608,-79.464763,Indie Alehouse,43.665475,-79.465290,Gastropub


### analyzing each neighbourhood

In [109]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Roselawn,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"High Park,The Junction South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"High Park,The Junction South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"High Park,The Junction South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"High Park,The Junction South",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
toronto_onehot.shape

(1686, 236)

### group rows by neighbourhood and by taking the mean of the frequency of occurrence of each category

In [113]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.000000,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown,St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012195,0.000000,...,0.000000,0.000000,0.00,0.012195,0.000000,0.000000,0.012195,0.000000,0.000000,0.012195
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.021739,0.000000,0.043478,0.010870,0.000000,0.000000,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011905,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.011905,0.000000,0.000000,0.011905,0.011905


In [114]:
toronto_grouped.shape

(38, 236)

### print each neighbourhood and its the top 5 most common venues

In [116]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2              Bar  0.04
3       Steakhouse  0.04
4  Thai Restaurant  0.04


----Berczy Park----
            venue  freq
0     Coffee Shop  0.07
1          Bakery  0.05
2  Farmers Market  0.04
3    Cocktail Bar  0.04
4        Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
                   venue  freq
0         Breakfast Spot  0.09
1                   Café  0.09
2  Performing Arts Venue  0.09
3                 Bakery  0.09
4            Coffee Shop  0.09


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2                Park  0.05
3          Comic Shop  0.05
4    Recording Studio  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0      Airport Lounge  0.13
1     Airport Servi

### sort the venues in descending order

In [117]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### resulting dataframe

In [119]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Cosmetics Shop,Asian Restaurant,Restaurant,Burger Joint,American Restaurant
1,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Cheese Shop,Beer Bar,Café,Steakhouse,Farmers Market,Seafood Restaurant,Breakfast Spot
2,"Brockton,Exhibition Place,Parkdale Village",Performing Arts Venue,Coffee Shop,Café,Breakfast Spot,Bakery,Intersection,Pet Store,Climbing Gym,Restaurant,Sandwich Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place,Brewery,Skate Park
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Harbor / Marina,Airport,Airport Food Court,Airport Gate,Boutique,Plane,Boat or Ferry


### cluster the neighbourhoods into 5 clusters

In [121]:
# set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

### add clustering labels to the dataframe

In [122]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Garden,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
1,M6P,West Toronto,"High Park,The Junction South",43.661608,-79.464763,3,Thai Restaurant,Bar,Mexican Restaurant,Café,Italian Restaurant,Discount Store,Bakery,Fried Chicken Joint,Cajun / Creole Restaurant,Diner
2,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,3,Café,Bar,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Bakery,Park,Burger Joint
3,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,3,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Seafood Restaurant,Steakhouse,Deli / Bodega,Bakery
4,M6S,West Toronto,"Runnymede,Swansea",43.651571,-79.484450,3,Coffee Shop,Café,Italian Restaurant,Pizza Place,Sushi Restaurant,Falafel Restaurant,Bar,Food & Drink Shop,Food,Fish & Chips Shop


In [124]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### examining each cluster

### cluster 1

In [125]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,East Toronto,0,Pub,Other Great Outdoors,Trail,Neighborhood,Pizza Place,Health Food Store,Event Space,Ethiopian Restaurant,Electronics Store,Dim Sum Restaurant
26,Central Toronto,0,Intersection,Gym,Tennis Court,Trail,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store
28,Central Toronto,0,Mexican Restaurant,Trail,Sushi Restaurant,Jewelry Store,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
35,Central Toronto,0,Food & Drink Shop,Park,Asian Restaurant,Clothing Store,Hotel,Sandwich Place,Gym,Breakfast Spot,Event Space,Ethiopian Restaurant


### cluster 2

In [126]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,1,Garden,Yoga Studio,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


### cluster 3

In [127]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Park,Swim School,Bus Line,Yoga Studio,Discount Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


### cluster 4

In [128]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,West Toronto,3,Thai Restaurant,Bar,Mexican Restaurant,Café,Italian Restaurant,Discount Store,Bakery,Fried Chicken Joint,Cajun / Creole Restaurant,Diner
2,Downtown Toronto,3,Café,Bar,Coffee Shop,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Dumpling Restaurant,Bakery,Park,Burger Joint
3,Downtown Toronto,3,Coffee Shop,Café,Hotel,Restaurant,American Restaurant,Gym,Seafood Restaurant,Steakhouse,Deli / Bodega,Bakery
4,West Toronto,3,Coffee Shop,Café,Italian Restaurant,Pizza Place,Sushi Restaurant,Falafel Restaurant,Bar,Food & Drink Shop,Food,Fish & Chips Shop
6,Downtown Toronto,3,Coffee Shop,Hotel,Aquarium,Café,Brewery,Scenic Lookout,Italian Restaurant,Fried Chicken Joint,Restaurant,Park
7,Downtown Toronto,3,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Bakery,Fast Food Restaurant,Restaurant,Japanese Restaurant,Tea Room
8,Downtown Toronto,3,Coffee Shop,Café,Bar,Thai Restaurant,Steakhouse,Cosmetics Shop,Asian Restaurant,Restaurant,Burger Joint,American Restaurant
9,Downtown Toronto,3,Coffee Shop,Bakery,Café,Pizza Place,Italian Restaurant,Restaurant,Pub,Park,Jewelry Store,Diner
10,Downtown Toronto,3,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Burger Joint,Ice Cream Shop,Bakery,Dessert Shop,Bar,Spa
11,East Toronto,3,Greek Restaurant,Coffee Shop,Ice Cream Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Restaurant,Grocery Store,Brewery,Bubble Tea Shop


### cluster 5

In [129]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,Downtown Toronto,4,Park,Playground,Trail,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
